In [ ]:
import os
import re
import shutil
from skimage import io
from PIL import Image

from torchvision import transforms
from torch.utils.data import DataLoader
from skimage import io
import torch
import glob
import torch
from torch import nn
from torchvision import models
from torch.optim import Adam
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import random
import torchvision
from torchvision import datasets, models, transforms

from itertools import cycle

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc
import copy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import glob
# import staintools
from PIL import ImageOps

import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
class BiopsyDataset100(torch.utils.data.Dataset):
    def __init__(self, root_dir, df, label, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.df = df
        self.image_filenames = []
        self.labels = []
        self.label = label
                
        image_dict = {}
        for image_filename in glob.glob(os.path.join(self.root_dir, '*.jpeg')):
            ID_number = image_filename.split('/')[-1].split('.tif')[0]  # Assuming your filenames are like SCAN1_01.jpg, SCAN1_02.jpg
            if ID_number in df.ID_number.tolist():
                if ID_number not in image_dict:
                    image_dict[ID_number] = []
                image_dict[ID_number].append(image_filename)

        random.seed(42)
        for ID_number, filenames in image_dict.items():
            chosen_filenames = random.sample(filenames, min(100, len(filenames)))
            for filename in chosen_filenames:
                self.image_filenames.append(filename)
                self.labels.append(df[df['ID_number'] == ID_number][self.label].values[0])

                
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image = io.imread(self.image_filenames[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label
    
    
    def get_labels(self):
        return self.labels


In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(128),
    transforms.ToTensor(),
])

In [ ]:
def get_dataset_continous(label, seed):
    df = pd.read_excel('data_file.xlsx')
    df = df[['ID_number', label]]
    df[label] = pd.to_numeric(df[label], errors='coerce')
    df = df.dropna(subset=[label])

    q20 = df[label].quantile(0.20)
    q80 = df[label].quantile(0.80)

    # Create new dataframe by filtering values below 20th percentile and above 80th percentile
    df_extreme = df[(df[label] <= q20) | (df[label] >= q80)].copy()

    # Set labels: 0 for values below or equal to 20th percentile, 1 for values above or equal to 80th percentile
    df_extreme.loc[df_extreme[label] <= q20, label] = 0
    df_extreme.loc[df_extreme[label] >= q80, label] = 1

    
    # Now sample the 5 images for each label
    np.random.seed(seed)
    sampled_df = df_extreme.groupby(label).apply(lambda x: x.sample(n=5)).reset_index(drop=True)
    return sampled_df

In [ ]:
# We first create an array that contains all the encoded images from the dataset: 
# And an array for all the labels:

def create_encoded_arr(dataset, encoder):
    
    encoded_arr = []
    labels = []
    
    for i in range(len(dataset)):
        image = dataset[i][0]
        label = dataset[i][1]
        
        encoded_arr.append(encoder(image))
        labels.append(label)

    return encoded_arr, labels

In [ ]:
def tsne(encoded_arr, labels):
    flattened_tensors = [tensor.detach().view(-1).numpy() for tensor in encoded_arr]
    data_array = np.stack(flattened_tensors)
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
    tsne_results = tsne.fit_transform(data_array)

    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels, cmap='viridis', alpha=0.7)
    unique_labels = np.unique(labels)
    colors = [plt.cm.viridis(label / max(unique_labels)) for label in unique_labels]
    legend_patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, 
                                 label=f'Label {int(label)}') for label, color in zip(unique_labels, colors)]
    plt.legend(handles=legend_patches, title='Labels', bbox_to_anchor=(0.75, 1), loc='upper left')
    plt.title('t-SNE Visualization of Encoded Images')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.show()

In [ ]:
# label = 'Ki67 (%)'
# label = 'TILS (%)'
# label = 'PR (%)'

# No encoder

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
        )

        self.decoder = nn.Sequential(   
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



In [ ]:
autoencoder = Autoencoder()

In [ ]:
encoder = autoencoder.encoder

In [ ]:
encoder

# 16384

In [ ]:
class Autoencoder_16K(nn.Module):
    def __init__(self):
        super(Autoencoder_16K, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),    
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
autoencoder_16K = Autoencoder_16K()

In [ ]:
autoencoder_16K.load_state_dict(torch.load('/storage/Chloe/final_model_autoencoder/autoencoder_64*16*16_split41.pth', map_location='cpu'))

In [ ]:
encoder_16K = autoencoder_16K.encoder

In [ ]:
encoder_16K

# 4096

In [ ]:
class Autoencoder_4K(nn.Module):
    def __init__(self):
        super(Autoencoder_4K, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),    
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
autoencoder_4K = Autoencoder_4K()

In [ ]:
autoencoder_4K.load_state_dict(torch.load('/storage/Chloe/final_model_autoencoder/autoencoder_256*4*4_split41.pth', map_location='cpu'))

In [ ]:
encoder_4K = autoencoder_4K.encoder

In [ ]:
encoder_4K

# 512

In [ ]:
class Autoencoder_512(nn.Module):
    def __init__(self):
        super(Autoencoder_512, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 4, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(4, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 4, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(4, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),    
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
autoencoder_512 = Autoencoder_512()

In [ ]:
autoencoder_512.load_state_dict(torch.load('/storage/Chloe/final_model_autoencoder/autoencoder_128*2*2_split41.pth', map_location='cpu'))

In [ ]:
encoder_512 = autoencoder_512.encoder

In [ ]:
encoder_512

# 64

In [ ]:
class Autoencoder_64(nn.Module):
    def __init__(self):
        super(Autoencoder_64, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 4, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(4, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 4, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(4, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),    
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
autoencoder_64 = Autoencoder_64()

In [ ]:
autoencoder_64.load_state_dict(torch.load('/storage/Chloe/final_model_autoencoder/autoencoder_64*1*1_split41.pth', map_location='cpu'))

In [ ]:
encoder_64 = autoencoder_64.encoder

In [ ]:
encoder_64

# 2 

In [ ]:
class Autoencoder_2(nn.Module):
    def __init__(self):
        super(Autoencoder_2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 4, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(4, 2, kernel_size=3, stride=2, padding=1),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(2, 4, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(4, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),    
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
autoencoder_2 = Autoencoder_2()

In [ ]:
autoencoder_2.load_state_dict(torch.load('/storage/Chloe/final_model_autoencoder/autoencoder_2*1*1_split41.pth', map_location='cpu'))

In [ ]:
encoder_2 = autoencoder_2.encoder

In [ ]:
encoder_2

# Training 

In [ ]:
label = 'Ki67 (%)'

In [ ]:
# No autoencoder
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder)
    tsne(encoded_arr, labels)

In [ ]:
# 16K 
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_16K)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 4K 
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_4K)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 512 
for i in range(10): 
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_512)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 64
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_64)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 2 
for i in range(10):
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_2)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
label =  'TILS (%)'

In [ ]:
# No autoencoder
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder)
    tsne(encoded_arr, labels)

In [ ]:
# 16K 
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_16K)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 4K 
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_4K)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 512 
for i in range(10): 
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_512)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 64
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_64)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 2 
for i in range(10):
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_2)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
label = 'PR (%)'

In [ ]:
# No autoencoder
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder)
    tsne(encoded_arr, labels)

In [ ]:
# 16K 
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_16K)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 4K 
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_4K)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 512 
for i in range(10): 
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_512)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 64
for i in range(10): 
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_64)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)

In [ ]:
# 2 
for i in range(10):
    print(f"Seed : {i}")
    sampled_df = get_dataset_continous(label, i)
    dataset = BiopsyDataset100("/storage/Chloe/zoom_20_512", sampled_df, label, transform=transform)
    encoded_arr, labels = create_encoded_arr(dataset, encoder_2)
    print(encoded_arr[0].shape)
    tsne(encoded_arr, labels)